<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import telebot
from telebot import types

import json

In [2]:
with open('./config.json', 'r') as f:
    json_config = json.load(f)

In [3]:
TOKEN = json_config['tg_bot_token']

bot = telebot.TeleBot(TOKEN, num_threads=4)

In [4]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-base")
model = T5ForConditionalGeneration.from_pretrained("../best_models/model_ruT5_5e-5_withPreprocessing_V1")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [5]:
def paraphrase(text, model, n=None, max_length='auto', temperature=0.0, beams=3):
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=temperature,
        repetition_penalty=3.0,
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [6]:
# Handle '/start' and '/help'
@bot.message_handler(commands=['start'])
def send_welcome(message):
    welcome_message = """Привет! Я бот-детоксификатор. Напиши мне какое-нибудь сообщение, и я попытаюсь его детоксифицировать."""
    bot.reply_to(message, welcome_message)

In [7]:
@bot.message_handler(commands=['help'])
def send_welcome(message):
    help_message = """В основе лежит модель ruT5-base (https://huggingface.co/ai-forever/ruT5-base), которую зафайнтьюнили для задачи детоксификации текстов на русском языке."""
    bot.reply_to(message, help_message)

In [8]:
@bot.message_handler(func=lambda message: True)
def paraphrase_message(message):
    detoxified = paraphrase(message.text, model)
    bot.reply_to(message, detoxified)

In [ ]:
bot.infinity_polling()